### NOTE: Change the paths for all the files, 
#### This work is done by  Pratyush sethi.

In [1]:
import pandas as pd
from urllib.request import Request, urlopen
import re
import time

In [2]:
# creating file url lists
df = pd.read_excel('C:/Users/Praty/OneDrive/Desktop/New folder/cik_list.xlsx')
url_list = []
for i in range(len(df['SECFNAME'])):
    url_list.append("https://www.sec.gov/Archives/" + df['SECFNAME'][i])

In [76]:
len(url_list)

152

In [4]:
# creating stopping word corpus
file_path = ['C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_Auditor.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_Currencies.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_DatesandNumbers.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_Generic.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_Geographic.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/StopWords_Names.txt']
stopword_corpus = []
for i in range(len(file_path)):
    with open(file_path[i],encoding="mbcs") as fp:
        for line in fp:
            line = line.lower()
            stopword_corpus.append(line.strip())
print(len(stopword_corpus))

# Making list of Master dictionary words
dict_df = pd.read_csv('C:/Users/Praty/OneDrive/Desktop/New folder/dict.csv')
dict_words = [x for x in dict_df['Word'].astype(str)]
for i in range(len(dict_words)):
    dict_words[i] = dict_words[i].lower()
    
path_file = ['C:/Users/Praty/OneDrive/Desktop/New folder/positive_words.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/negative_words.txt',
             'C:/Users/Praty/OneDrive/Desktop/New folder/constraining_dictionary.xlsx',
             'C:/Users/Praty/OneDrive/Desktop/New folder/uncertainty_dictionary.xlsx']
# making positive word list
positive_words = []
with open(path_file[0], encoding="mbcs") as fp:
        for line in fp:
            line = line.lower()
            positive_words.append(line.strip())
            
# making negative word list
negative_words = []
with open(path_file[1], encoding="mbcs") as fp:
        for line in fp:
            line = line.lower()
            negative_words.append(line.strip())
# making uncertainity word list
con_dict = pd.read_excel(path_file[2])
con_list = [x for x in con_dict['Word']]
# making constraining word list
un_dict = pd.read_excel(path_file[3])
un_list = [x for x in un_dict['Word']]

# complex word list
complex_word_list = []
for i in range(len(dict_df['Word'])):
    if dict_df['Syllables'][i] > 2:
        complex_word_list.append(dict_df['Word'][i])


13536


In [10]:
positive_score = []
negative_score = []
polarity_score = []
avg_sen_len_1 = []
per_of_complex_words = []             
fog_index = []
complex_word = []
word_count = []
uncertainty_score = []
constraining_score = []
positive_word_proportion = []
negative_word_proportion = []
un_word_proportion = []
con_word_proportion = []
con_words_whole_report = [] 

In [133]:
def tokenizing_sentence(sentence):
    sentence = sentence.lower()
    sentence = [tok for tok in sentence.split()]
    return sentence

def open_files(url):
    data = []
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    request = Request(url, headers = hdr)
    file = urlopen(request)
    for line in file:
        decoded_line = line.decode("utf-8")
        data.append(decoded_line)
        
    # tokenizing the sentences
    processed_sen = [tokenizing_sentence(sentence) for sentence in data]
    # number of sentences
    num_sen = len(processed_sen)
    # removing stopwords and other preprocessing
    clean_txt = []
    sw_found = 0
    for i in range(len(processed_sen)):
        for j in range(len(processed_sen[i])):
            if not processed_sen[i][j] in stopword_corpus:
                clean_txt.append(processed_sen[i][j])
            else:
                sw_found +=1
    # removing html text and integers this is just for dowsizing the words size
    cleaner_txt =[]
    no_integers = []
    clean = re.compile(r'<.*?>')
    for i in range(len(clean_txt)):
        cleaner_txt.append(re.sub(clean,'',clean_txt[i]))
    no_integers = [x for x in cleaner_txt if not (x.isdigit())]  
                       
    # extracting words by matching them from master dictionary
    final_list = []
    for i in range(len(no_integers)):
        if no_integers[i] in dict_words:
            final_list.append(no_integers[i])
    # number of words             
    total_words = len(final_list)
    
    # positive words counting                   
    positive_count = 0
    for i in range(len(final_list)):
        if final_list[i] in positive_words:
            positive_count +=1                   
    # negative word counting
    negative_count = 0
    for i in range(len(final_list)):
        if final_list[i] in negative_words:
            negative_count +=1
    # complex word count
    complex_words_count = 0                   
    for i in range(len(final_list)):
        if final_list[i] in complex_word_list:
            complex_words_count +=1
    # constraining words count
    con = 0
    for i in range(len(final_list)):
        if final_list[i] in con_list:
            con +=1
    # uncertainity words count
    un = 0
    for i in range(len(final_list)):
        if final_list[i] in un_list:
            un +=1
    # constraining words in whole report
    con_words_count = 0
    for i in range(len(no_integers)):
        if no_integers[i] in con_list:
            con_words_count +=1
            
    # calculating fifteen terms
    positive_score.append(positive_count) #1
    negative_score.append(negative_count) #2
    polarity_score.append((positive_count - negative_count)/(positive_count + negative_count) + 0.000001) #3
    avg_sen_len = total_words/len(processed_sen)
    avg_sen_len_1.append(total_words /len(processed_sen)) #4
    complex_count_word_per = complex_words_count/total_words
    per_of_complex_words.append(complex_words_count/ total_words) #5             
    fog_index.append(0.4*(avg_sen_len + complex_count_word_per)) #6
    complex_word.append(complex_words_count) #7
    word_count.append(total_words) #8
    uncertainty_score.append(un) #9
    constraining_score.append(con) #10
    positive_word_proportion.append(positive_count/ total_words) #11
    negative_word_proportion.append(negative_count/ total_words) #12
    un_word_proportion.append(un/ total_words) #13
    con_word_proportion.append(con/ total_words) #14
    con_words_whole_report.append(con_words_count) #15
    
for i in range(146,len(url_list)):
    start_time = time.time()
    open_files(url_list[i])
    print(i,"Completed","--- %s seconds ---" % (time.time() - start_time))

146 Completed --- 4.009963750839233 seconds ---
147 Completed --- 129.4397416114807 seconds ---
148 Completed --- 5.168937921524048 seconds ---
149 Completed --- 26.16189742088318 seconds ---
150 Completed --- 120.62368988990784 seconds ---
151 Completed --- 44.74918222427368 seconds ---


In [136]:
# creating table
table = pd.DataFrame({"CIK":df['CIK'],
                     "CONAME":df['CONAME'],
                     "FYRMO":df['FYRMO'],
                     "FROM":df['FDATE'],
                     "SECFNAME":df['FORM'],
                     "positive_score":positive_score,
                     "negative_score":negative_score,
                     "polarity_score":polarity_score,
                     "average_sentence_length":avg_sen_len_1,
                     "percentage_of_complex_words":per_of_complex_words,
                     "fog_index":fog_index,
                     "complex_word_count":complex_word,
                     "word_count":word_count,
                     "uncertainty_score":uncertainty_score,
                     "constraining_score":constraining_score,
                     "positive_word_proportion":positive_word_proportion,
                     "negative_word_proportion":negative_word_proportion,
                     "uncertainty_word_proportion":un_word_proportion,
                     "constraining_word_proportion":con_word_proportion,
                     "constraining_words_Whole_report":con_words_whole_report})

In [137]:
table.head(10)

,CIK,CONAME,FYRMO,FROM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainty_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainty_word_proportion,constraining_word_proportion,constraining_words_Whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,3097,1587,0.322375,2.763135,0.0,1.105254,0,63530,0,0,0.048749,0.024980,0.0,0.0,0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,2015,864,0.399793,3.036022,0.0,1.214409,0,42225,0,0,0.047721,0.020462,0.0,0.0,0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,9,4,0.384616,1.502439,0.0,0.600976,0,308,0,0,0.029221,0.012987,0.0,0.0,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,1277,801,0.229067,2.554894,0.0,1.021957,0,30881,0,0,0.041352,0.025938,0.0,0.0,0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,12,4,0.500001,1.714286,0.0,0.685714,0,384,0,0,0.031250,0.010417,0.0,0.0,0
5,3662,SUNBEAM CORP/FL/,199811,1998-11-25,10-Q/A,196,227,-0.073285,2.748192,0.0,1.099277,0,6079,0,0,0.032242,0.037342,0.0,0.0,0
6,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,408,418,-0.012106,2.688989,0.0,1.075596,0,10159,0,0,0.040161,0.041146,0.0,0.0,0
7,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,195,230,-0.082352,2.795918,0.0,1.118367,0,6302,0,0,0.030943,0.036496,0.0,0.0,0
8,3662,SUNBEAM CORP/FL/,199903,1999-03-31,NT 10-K,10,4,0.428572,1.619910,0.0,0.647964,0,358,0,0,0.027933,0.011173,0.0,0.0,0
9,3662,SUNBEAM CORP/FL/,199905,1999-05-11,10-K,1351,855,0.224842,2.920590,0.0,1.168236,0,31262,0,0,0.043215,0.027349,0.0,0.0,0


In [138]:
table.to_csv('C:/Users/Praty/OneDrive/Desktop/New folder/output_table.csv')

In [135]:
print(len(positive_score))
print(len(negative_score))
print(len(polarity_score))
print(len(avg_sen_len_1))
print(len(per_of_complex_words))             
print(len(fog_index))
print(len(complex_word))
print(len(word_count))
print(len(uncertainty_score))
print(len(constraining_score))
print(len(positive_word_proportion))
print(len(negative_word_proportion))
print(len(un_word_proportion))
print(len(con_word_proportion))
print(len(con_words_whole_report))

152
152
152
152
152
152
152
152
152
152
152
152
152
152
152
